In [13]:
import sklearn.preprocessing as preprocess
import numpy as np
import pandas as pd
from keras import backend as K
import spacy
from spacy.lang.en import English
pd.set_option('display.max_colwidth', 0)

In [2]:
df_hd_tp = pd.read_csv('evaluation_set/headline_topic_mapping.csv')
df_ar_cl = pd.read_csv('evaluation_set/article_claim_mapping.csv')
df_hd_tp.keys()

Index(['Topic', 'Title', 'article Id', 'Headline'], dtype='object')

In [3]:
hd = df_hd_tp[df_hd_tp.Title=='Video game controversies']['Headline'].values[0]
ar_id = df_hd_tp[df_hd_tp.Title=='Video game controversies']['article Id'].values[0]
cl = df_ar_cl[df_ar_cl.Article=='Video game controversies']['Claim'].values
print('ar_id:',ar_id)
print('hd:',hd)
print('cl_1:',cl[0])


ar_id: 1
hd: The sale of violent video games to minors should be banned
cl_1: exposure to violent video games causes at least a temporary increase in aggression and that this exposure correlates with aggression in the real world


In [4]:
with open('evaluation_set/articles/clean_{0}.txt'.format(ar_id)) as f:
    article = f.readlines()
# sentences = article.split('\n')
article=''.join(article)
# print(article)
article = article.replace('[REF]','')
nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer')) # updated
doc = nlp(article)
sentences = [sent.string.strip() for sent in doc.sents]
len(sentences)

266

# Load spacy vector library

In [5]:
nlp = spacy.load("en_core_web_md")

# Spacy internal sentence vector
* get best N similar with headline

In [52]:
tokens_hd = nlp(hd)
sims = []
for i in range(len(sentences)):
    tokens_s = nlp(sentences[i])
    sims.append(tokens_hd.similarity(tokens_s))
best_N = 50
top_N = []
for idx in np.array(sims).argsort()[-best_N:][::-1]:
    top_N.append(sentences[idx])
    print(sentences[idx],'sim:',np.around(sims[idx],4))
    print('----')

The case centered on a California law that sought to restrict sales of violent video games to minors. sim: 0.9245
----
Certain game publishers’ decision to have controversial games rated seems to show that they are not targeted at young children. sim: 0.9049
----
The adult demographic is the fastest-growing segment of the American video games market with 32% of adults playing video games, although critics have suggested that such statistics are often used to deflect from the fact that almost all American children are exposed to video games . sim: 0.9008
----
He argues that video game publishers unethically train children in the use of weapons and, more importantly, harden them emotionally to the act of murder by simulating the killing of hundreds or thousands of opponents in a single typical video game. sim: 0.899
----
No video game console manufacturer has yet to allow any game to be published in North America with the harshest ESRB rating, "Adults Only", signaling that the game is on

# Similarity between Claims and Top N

In [53]:
threshold = 0.94
pred_counter = 0
pred_similarity = []
ground_truth_matching_sent = []
pred_claim_sent = []
for i in range(len(top_N)):
#     print('===========***********',i,'***********============')
    t5 = nlp(top_N[i])
    pred_claim_sent.append(top_N[i])
#     print(t5.vector)
    for j in range(len(cl)):
        _c = nlp(cl[j])
#         print(_c.vector)
#         print('top_5:',t5.text)
#         print('-------------------')
#         print('ground_truth:',_c.text)
#         print('t5:{0}, cl:{1}, sim: {2}'.format(i,j,np.around(t5.similarity(_c),4)))
#         print('===================================================================')
        if np.around(t5.similarity(_c),4) > threshold:
            ground_truth_matching_sent.append(cl[j])
            pred_similarity.append(np.around(t5.similarity(_c),4))
            pred_counter+=1
            break
    if len(ground_truth_matching_sent)!=len(pred_claim_sent):
        ground_truth_matching_sent.append('None')
        pred_similarity.append(0.0)
            
df = pd.DataFrame()
df['Headline'] = np.repeat(hd,len(top_N))
df['pred_claim_sent'] = pred_claim_sent
df['ground_truth_matching_sent'] = ground_truth_matching_sent
df['pred_similarity'] = pred_similarity
print(pred_counter)
df
        
        
        
    

6


,Headline,pred_claim_sent,ground_truth_matching_sent,pred_similarity
0,The sale of violent video games to minors should be banned,The case centered on a California law that sought to restrict sales of violent video games to minors.,None,0.0000
1,The sale of violent video games to minors should be banned,Certain game publishers’ decision to have controversial games rated seems to show that they are not targeted at young children.,adolescents that don't play video games at all are most at-risk for violent behavior,0.9409
2,The sale of violent video games to minors should be banned,"The adult demographic is the fastest-growing segment of the American video games market with 32% of adults playing video games, although critics have suggested that such statistics are often used to deflect from the fact that almost all American children are exposed to video games .",None,0.0000
3,The sale of violent video games to minors should be banned,"He argues that video game publishers unethically train children in the use of weapons and, more importantly, harden them emotionally to the act of murder by simulating the killing of hundreds or thousands of opponents in a single typical video game.","video game publishers unethically train children in the use of weapons and, more importantly, harden them emotionally to the act of murder",0.9793
4,The sale of violent video games to minors should be banned,"No video game console manufacturer has yet to allow any game to be published in North America with the harshest ESRB rating, ""Adults Only"", signaling that the game is only appropriate for ages 18 years and up.",None,0.0000
5,The sale of violent video games to minors should be banned,"In the US, ESRB ratings are not legally binding, but many retailers take it upon themselves to refuse the sale of these games to minors.",None,0.0000
6,The sale of violent video games to minors should be banned,Dr. Olson has come to the conclusion that violent games affect students positively and not negatively because the violent crime rate is going down while the popularity of M-rated video games has increased.,None,0.0000
7,The sale of violent video games to minors should be banned,"In the decision, the Court determined that there was no direct link between violent video games and their influence on children, in as much as compared to other types of media like television and pictures, and that they could not create a new class of restricted speech that is not applied to other forms of media .",there was no direct link between violent video games and their influence on children,0.9439
8,The sale of violent video games to minors should be banned,Sexual themes in video games are much less tolerated in the US than violent themes; the most well-known example of this is the Hot Coffee controversy.,None,0.0000
9,The sale of violent video games to minors should be banned,"During heights of video game controversy he has been interviewed on the content of his books, and has repeatedly used the term ""murder simulator"" to describe first-person shooter games.",None,0.0000


In [54]:
df_results = pd.DataFrame()
df_results['precision'] = [pred_counter/best_N]
df_results['recall'] = [pred_counter/len(cl)]
df_results['f1'] = [2*precision*recall/(precision+recall)]
# print('Recall:{0}, Precision:{1}, F_1:{2}'.format(np.around(recall,4),np.around(precision,4),np.around(f1,4)))
df_results = df_results.T
df_results.columns = ['@{0}'.format(best_N)]
df_results

,@50
precision,0.120000
recall,0.187500
f1,0.142857


In [55]:
pred_counter

6

In [ ]:
# precision at 50 is for ranking: the extracted entences are ranked and checked if the order is maintained as ground truth: useless time waste
3/10 whose similarity is > 0.95, 
precision = relevant retrived = 3/total retrieved = 10
recall = relevant retrived = 3/total ground truth = 22
f1 = 2pr/(p+r)

In [8]:
# def get_sent_emb(token_vectors,mode='sum'):
#     if mode == 'sum':
#         return np.sum(token_vectors)

# tokens_hd = nlp(hd)
# # hd_emb = get_sent_emb(tokens_hd.)
# sims = []
# for i in range(len(sentences)):#     print(s)
#     tokens_s = nlp(sentences[i])
# #     print('sim {0}:{1}'.format(i,tokens_hd.similarity(tokens_s)))
#     sims.append(tokens_hd.similarity(tokens_s))

    
# # tokens_s.vector
# best_count = 5
# # print(np.array(sims).argsort()[-best_count:][::-1])
# for idx in np.array(sims).argsort()[-best_count:][::-1]:
#     print(sentences[idx],'sim:',np.around(sims[idx],4))
#     print('----')

In [9]:
# import keras
# from keras_bert import get_base_dict, get_model, compile_model, gen_batch_inputs


# # A toy input example
# sentence_pairs = [
#     [['all', 'work', 'and', 'no', 'play'], ['makes', 'jack', 'a', 'dull', 'boy']],
#     [['from', 'the', 'day', 'forth'], ['my', 'arm', 'changed']],
#     [['and', 'a', 'voice', 'echoed'], ['power', 'give', 'me', 'more', 'power']],
# ]


# # Build token dictionary
# token_dict = get_base_dict()  # A dict that contains some special tokens
# for pairs in sentence_pairs:
#     for token in pairs[0] + pairs[1]:
#         if token not in token_dict:
#             token_dict[token] = len(token_dict)
# token_list = list(token_dict.keys())  # Used for selecting a random word

# def _custom_layers(x, trainable=True):
#     return keras.layers.LSTM(
#         units=768,
#         trainable=trainable,
#         name='LSTM',
#     )(x)


# # Build & train the model
# model = get_model(
#     token_num=len(token_dict),
#     head_num=5,
#     transformer_num=12,
#     embed_dim=25,
#     feed_forward_dim=100,
#     seq_len=20,
#     pos_num=20,
#     dropout_rate=0.05,
#     custom_layers=_custom_layers,
# )
# compile_model(model)
# model.summary()

# def _generator():
#     while True:
#         yield gen_batch_inputs(
#             sentence_pairs,
#             token_dict,
#             token_list,
#             seq_len=20,
#             mask_rate=0.3,
#             swap_sentence_rate=1.0,
#         )

# # model.fit_generator(
# #     generator=_generator(),
# #     steps_per_epoch=1000,
# #     epochs=100,
# #     validation_data=_generator(),
# #     validation_steps=100,
# #     callbacks=[
# #         keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# #     ],
# # )


# # # Use the trained model
# # inputs, output_layer = get_model(
# #     token_num=len(token_dict),
# #     head_num=5,
# #     transformer_num=12,
# #     embed_dim=25,
# #     feed_forward_dim=100,
# #     seq_len=20,
# #     pos_num=20,
# #     dropout_rate=0.05,
# #     training=False,      # The input layers and output layer will be returned if `training` is `False`
# #     trainable=False,     # Whether the model is trainable. The default value is the same with `training`
# #     output_layer_num=4,  # The number of layers whose outputs will be concatenated as a single output.
# #                          # Only available when `training` is `False`.
# # )

In [10]:
# inputs

In [11]:
# output_layer

In [12]:
# next(_generator())

In [13]:
# np.shape(x)

In [14]:
# y

In [15]:
# token_dict

In [16]:
# token_list

In [17]:
# # Build token dictionary
# token_dict = get_base_dict()  # A dict that contains some special tokens
# for pairs in sentence_pairs:
#     for token in pairs[0] + pairs[1]:
#         if token not in token_dict:
#             token_dict[token] = len(token_dict)

In [18]:
# model.predict(_generator())

In [39]:
np.repeat(hd,10)

array(['The sale of violent video games to minors should be banned',
       'The sale of violent video games to minors should be banned',
       'The sale of violent video games to minors should be banned',
       'The sale of violent video games to minors should be banned',
       'The sale of violent video games to minors should be banned',
       'The sale of violent video games to minors should be banned',
       'The sale of violent video games to minors should be banned',
       'The sale of violent video games to minors should be banned',
       'The sale of violent video games to minors should be banned',
       'The sale of violent video games to minors should be banned'],
      dtype='<U58')